In [1]:
import sys
from math import pi
from pathlib import Path

import numpy as np
import pygfunction as gt
import tabulate

In [2]:
here = Path()
here.resolve()

PosixPath('/home/mitchute/Projects/BHResist/notebooks')

In [3]:
root = here / ".."
root.exists()

True

In [4]:
sys.path.insert(0, str(root))

In [5]:
from bhr.borehole import Borehole

# Comparison - PyGFunction

pygfunction example code taken from [here](https://github.com/MassimoCimmino/pygfunction/blob/9217d8d77c124239d4c1c881e34eb44c9ac6ade4/examples/custom_borehole.py)

## Single U-tube

In [6]:
# -------------------------------------------------------------------------
# Define a single U-tube borehole
# -------------------------------------------------------------------------

# Borehole dimensions
D = 5.0  # Borehole buried depth (m)
H = 400.0  # Borehole length (m)
r_b = 0.0875  # Borehole radius (m)

# Pipe dimensions (all configurations)
epsilon = 1.0e-6  # Pipe roughness (m)

# Pipe dimensions (single U-tube and double U-tube)
r_out = 0.0211  # Pipe outer radius (m)
r_in = 0.0147  # Pipe inner radius (m)
D_s = 0.052  # Shank spacing (m)

# Ground properties
k_s = 2.0  # Ground thermal conductivity (W/m.K)

# Grout properties
k_g = 1.0  # Grout thermal conductivity (W/m.K)

# Pipe properties
k_p = 0.4  # Pipe thermal conductivity (W/m.K)

# Fluid properties
# Total fluid mass flow rate per borehole (kg/s)
m_flow_borehole = 1.0

# The fluid is propylene-glycol (20 %) at 20 degC
fluid = gt.media.Fluid("MPG", 20.0)
cp_f = fluid.cp  # Fluid specific isobaric heat capacity (J/kg.K)
rho_f = fluid.rho  # Fluid density (kg/m3)
mu_f = fluid.mu  # Fluid dynamic viscosity (kg/m.s)
k_f = fluid.k  # Fluid thermal conductivity (W/m.K)

# -------------------------------------------------------------------------
# Initialize borehole model
# -------------------------------------------------------------------------

borehole = gt.boreholes.Borehole(H, D, r_b, x=0.0, y=0.0)

# -------------------------------------------------------------------------
# Define a single U-tube borehole
# -------------------------------------------------------------------------

# Pipe positions
# Single U-tube [(x_in, y_in), (x_out, y_out)]
pos_single = [(-D_s, 0.0), (D_s, 0.0)]

# Pipe thermal resistance
R_p = gt.pipes.conduction_thermal_resistance_circular_pipe(r_in, r_out, k_p)

# Fluid to inner pipe wall thermal resistance
m_flow_pipe = m_flow_borehole
h_f = gt.pipes.convective_heat_transfer_coefficient_circular_pipe(m_flow_pipe, r_in, mu_f, rho_f, k_f, cp_f, epsilon)
R_f = 1.0 / (h_f * 2 * pi * r_in)

# Single U-tube GHE in borehole
SingleUTube = gt.pipes.SingleUTube(pos_single, r_in, r_out, borehole, k_s, k_g, R_f + R_p)

# Evaluate and print the effective borehole thermal resistance
R_b = SingleUTube.effective_borehole_thermal_resistance(m_flow_borehole, fluid.cp)

In [7]:
single_bhr = Borehole()
single_bhr.init_single_u_borehole(
    borehole_diameter=r_b * 2,
    pipe_outer_diameter=r_out * 2,
    pipe_dimension_ratio=(2 * r_out) / (r_out - r_in),
    length=H,
    shank_space=D_s,
    pipe_conductivity=k_p,
    grout_conductivity=k_g,
    soil_conductivity=k_s,
    fluid_type="PROPYLENEGLYCOL",
    fluid_concentration=0.2,
)

In [8]:
temp = 20

single_data = [
    ["", "pygfunction", "BHResist"],
    ["Borehole Dia.", r_b * 2, single_bhr._bh.borehole_diameter],
    ["Pipe Outer Dia.", r_out * 2, single_bhr._bh.pipe_outer_diameter],
    ["Pipe Inner Dia.", r_in * 2, single_bhr._bh.pipe_inner_diameter],
    ["Shank Spacing", D_s, single_bhr._bh.shank_space],
    ["Rp", f"{R_p:0.5f}", f"{single_bhr._bh.calc_pipe_cond_resist():0.5f}"],
    ["Rf", f"{R_f:0.5f}", f"{single_bhr._bh.calc_pipe_internal_conv_resist(m_flow_borehole, temp):0.5f}"],
    ["Rb", f"{R_b:0.5f}", f"{single_bhr.calc_bh_resist(m_flow_borehole, temp):0.5f}"],
]

table_single = tabulate.tabulate(single_data, tablefmt="html")
table_single

,pygfunction,BHResist
Borehole Dia.,0.175,0.175
Pipe Outer Dia.,0.0422,0.0422
Pipe Inner Dia.,0.0294,0.0294
Shank Spacing,0.052,0.052
Rp,0.14381,0.14381
Rf,0.00298,0.00297
Rb,0.17376,0.17377


## Double U-tube 

In [9]:
# -------------------------------------------------------------------------
# Define a double U-tube borehole
# -------------------------------------------------------------------------

# Pipe positions
# Double U-tube [(x_in1, y_in1), (x_in2, y_in2),
#                (x_out1, y_out1), (x_out2, y_out2)]
# Note: in series configuration, fluid enters pipe (in,1), exits (out,1),
# then enters (in,2) and finally exits (out,2)
# (if you view visualize_pipe, series is 1->3->2->4)
pos_double = [(-D_s, 0.), (0., -D_s), (D_s, 0.), (0., D_s)]

# Pipe thermal resistance
R_p = gt.pipes.conduction_thermal_resistance_circular_pipe(
        r_in, r_out, k_p)

# Fluid to inner pipe wall thermal resistance

# Double U-tube in parallel
m_flow_pipe_parallel = m_flow_borehole / 2
h_f_parallel = gt.pipes.convective_heat_transfer_coefficient_circular_pipe(
    m_flow_pipe_parallel, r_in, mu_f, rho_f, k_f, cp_f, epsilon)
R_f_parallel = 1.0 / (h_f_parallel * 2 * pi * r_in)

# Double U-tube in parallel
DoubleUTube_parallel = gt.pipes.MultipleUTube(
    pos_double, r_in, r_out, borehole, k_s, k_g, R_p + R_f_parallel, 2,
    config='parallel')

# Evaluate and print the effective borehole thermal resistance
R_b_parallel = DoubleUTube_parallel.effective_borehole_thermal_resistance(
    m_flow_borehole, fluid.cp)

In [10]:
double_bhr = Borehole()
double_bhr.init_double_u_borehole(
    borehole_diameter=r_b * 2,
    pipe_outer_diameter=r_out * 2,
    pipe_dimension_ratio=(2 * r_out) / (r_out - r_in),
    length=H,
    shank_space=D_s,
    pipe_conductivity=k_p,
    pipe_inlet_arrangement="DIAGONAL",
    grout_conductivity=k_g,
    soil_conductivity=k_s,
    fluid_type="PROPYLENEGLYCOL",
    fluid_concentration=0.2,
)

In [11]:
double_data = [
    ["", "pygfunction", "BHResist"],
    ["Borehole Dia.", r_b * 2, double_bhr._bh.borehole_diameter],
    ["Pipe Outer Dia.", r_out * 2, double_bhr._bh.pipe_outer_diameter],
    ["Pipe Inner Dia.", r_in * 2, double_bhr._bh.pipe_inner_diameter],
    ["Shank Spacing", D_s, double_bhr._bh.shank_space],
    ["Rp", f"{R_p:0.5f}", f"{double_bhr._bh.calc_pipe_cond_resist():0.5f}"],
    ["Rf", f"{R_f_parallel:0.5f}", f"{double_bhr._bh.calc_pipe_internal_conv_resist(m_flow_pipe_parallel, temp):0.5f}"],
    ["Rb", f"{R_b_parallel:0.5f}", f"{double_bhr.calc_bh_resist(m_flow_borehole, temp):0.5f}"],
]

double_single = tabulate.tabulate(double_data, tablefmt="html")
double_single

,pygfunction,BHResist
Borehole Dia.,0.175,0.175
Pipe Outer Dia.,0.0422,0.0422
Pipe Inner Dia.,0.0294,0.0294
Shank Spacing,0.052,0.052
Rp,0.14381,0.14381
Rf,0.00566,0.00560
Rb,0.10933,0.10248


In [13]:
(double_bhr.calc_bh_resist(m_flow_borehole, temp) - R_b_parallel	) / R_b_parallel

-0.06272240516216238